NvidiaRacecarのインスタンスを作成します。

In [ ]:
from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar()

操作用のWidgetを作成します。ラジコンの3chのスイッチを押し、Jetson Modeに切り替えます(LEDが赤色っぽくなります。)

本Widgetは、イベントを登録シて初めて有効になります。必ず、次のプログラムも実行してください。

In [ ]:
import ipywidgets.widgets as widgets
from IPython.display import display

# create two sliders with range [-1.0, 1.0]
throttle_slider = widgets.FloatSlider(description='throttle', min=-1.0, max=1.0, step=0.01, orientation='vertical')
steering_slider = widgets.FloatSlider(description='steering', min=-1.0, max=1.0, step=0.01, orientation='horizontal')
steering_gain = widgets.BoundedFloatText(description='st_gain',min=-1.0, max=1.0, step=0.01, value=car.steering_gain)
steering_offset = widgets.BoundedFloatText(description='st_offset',min=-1.0, max=1.0, step=0.01, value=car.steering_offset)
throttle_gain = widgets.BoundedFloatText(description='th_gain',min=-1.0, max=1.0, step=0.01, value=car.throttle_gain)

# create a horizontal box container to place the sliders next to eachother
slider_container = widgets.HBox([throttle_slider, throttle_gain])
value_container = widgets.HBox([steering_slider,steering_gain,steering_offset])

# display the container in this cell's output
display(slider_container)
display(value_container)

In [ ]:
import traitlets

throttle_link = traitlets.link((throttle_slider, 'value'), (car, 'throttle'))
steering_link = traitlets.link((steering_slider, 'value'), (car, 'steering'))
steering_gain_link = traitlets.link((steering_gain, 'value'), (car, 'steering_gain'))
throttle_gain_link = traitlets.link((throttle_gain, 'value'), (car, 'throttle_gain'))
steering_offset_link = traitlets.link((steering_offset, 'value'), (car, 'steering_offset'))

st_gain(steering gain), st_offset(steering offset) の値をメモしておきます。